# Intro notebook
The purpose of this notebook is to connect to a local database that we created by loading CSV files into that database.
Copy / paste this notebook into other places to add / delete cells and add new analysis.

In [100]:
import os
from pathlib import Path
from sqlalchemy import create_engine
import pandas as pd
from collections import deque

In [17]:
# Define your database file path (SQLite example)
rel_db_path = Path.cwd().parent.parent / 'raw_data' / 'nfl' / 'src_code' / 'nfl.db'
db_file_path = rel_db_path.resolve()
engine = create_engine(f'sqlite:///{db_file_path}')
print(engine)

Engine(sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db)


In [18]:
%load_ext sql

In [19]:
%sql sqlite:///{db_file_path}

In [60]:
%%sql
SELECT
    fantasy_rank_pos,
    AVG(IIF(fantasy_pos = 'QB',fppg,NULL)) AS QB,
    GROUP_CONCAT(IIF(fantasy_pos = 'QB',cleaned_player,NULL)) AS QB_NAME,
    AVG(IIF(fantasy_pos = 'RB',fppg,NULL)) AS RB,
    GROUP_CONCAT(IIF(fantasy_pos = 'RB',cleaned_player,NULL)) AS RB_NAME,
    AVG(IIF(fantasy_pos = 'WR',fppg,NULL)) AS WR,
    GROUP_CONCAT(IIF(fantasy_pos = 'WR',cleaned_player,NULL)) AS WR_NAME,
    AVG(IIF(fantasy_pos = 'TE',fppg,NULL)) AS TE,
    GROUP_CONCAT(IIF(fantasy_pos = 'TE',cleaned_player,NULL)) AS TE_NAME
FROM (SELECT * FROM nfl_enhanced ORDER BY fppg_ppr DESC)
WHERE 1=1
    AND year = '2023'
    AND fantasy_rank_pos < 40
GROUP BY 1
LIMIT 10

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


fantasy_rank_pos,QB,QB_NAME,RB,RB_NAME,WR,WR_NAME,TE,TE_NAME
1,23.12,Josh Allen,20.25,Christian McCaffrey,15.76,CeeDee Lamb,9.0,Sam LaPorta
2,21.0,Jalen Hurts,16.2,Raheem Mostert,16.06,Tyreek Hill,8.63,George Kittle
3,20.18,Dak Prescott,13.18,Travis Etienne,13.25,Amon-Ra St. Brown,8.4,Travis Kelce
4,20.69,Lamar Jackson,18.58,Kyren Williams,12.0,Mike Evans,8.27,T.J. Hockenson
5,18.76,Jordan Love,12.88,Derrick Henry,11.41,Puka Nacua,7.5,David Njoku
6,18.5,Brock Purdy,12.65,Joe Mixon,11.24,D.J. Moore,6.82,Evan Engram
7,17.0,Jared Goff,12.65,Breece Hall,12.27,Deebo Samuel,6.94,Taysom Hill
8,17.5,Patrick Mahomes,12.0,Rachaad White,10.82,A.J. Brown,6.35,Cole Kmet
9,18.33,C.J. Stroud,13.64,David Montgomery,12.0,Nico Collins,6.24,Jake Ferguson
10,16.12,Baker Mayfield,12.67,Jahmyr Gibbs,10.88,Brandon Aiyuk,5.94,Trey McBride


In [96]:
vectors = """
SELECT
    year AS YEAR,
    fantasy_rank_pos,
    AVG(IIF(fantasy_pos = 'QB',fantasy_points,NULL)) AS QB,
    GROUP_CONCAT(IIF(fantasy_pos = 'QB',cleaned_player,NULL)) AS QB_NAME,
    AVG(IIF(fantasy_pos = 'RB',fantasy_points,NULL)) AS RB,
    GROUP_CONCAT(IIF(fantasy_pos = 'RB',cleaned_player,NULL)) AS RB_NAME,
    AVG(IIF(fantasy_pos = 'WR',fantasy_points,NULL)) AS WR,
    GROUP_CONCAT(IIF(fantasy_pos = 'WR',cleaned_player,NULL)) AS WR_NAME,
    AVG(IIF(fantasy_pos = 'TE',fantasy_points,NULL)) AS TE,
    GROUP_CONCAT(IIF(fantasy_pos = 'TE',cleaned_player,NULL)) AS TE_NAME
FROM nfl_enhanced
WHERE 1=1
    AND fantasy_rank_pos < 40
GROUP BY 1, 2
"""

In [97]:
%%sql
WITH vect AS ($vectors)
SELECT * FROM vect
WHERE 1=1
    AND fantasy_rank_pos <= 10
LIMIT 30

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


YEAR,fantasy_rank_pos,QB,QB_NAME,RB,RB_NAME,WR,WR_NAME,TE,TE_NAME
2016,1,380.0,Aaron Rodgers,328.0,David Johnson,208.0,Mike Evans,138.0,Travis Kelce
2016,2,347.0,Matt Ryan,293.0,Ezekiel Elliott,208.0,Jordy Nelson,127.0,Greg Olsen
2016,3,332.0,Drew Brees,248.0,LeSean McCoy,201.0,Antonio Brown,126.0,Kyle Rudolph
2016,4,308.0,Andrew Luck,242.0,Le'Veon Bell,196.0,Odell Beckham Jr.,124.0,Jimmy Graham
2016,5,300.0,Kirk Cousins,241.0,DeMarco Murray,183.0,T.Y. Hilton,123.0,Delanie Walker
2016,6,287.0,Dak Prescott,230.0,Devonta Freeman,177.0,Julio Jones,114.0,Cameron Brate
2016,7,280.0,Matthew Stafford,226.0,LeGarrette Blount,172.0,Davante Adams,113.0,Martellus Bennett
2016,8,271.0,Tyrod Taylor,210.0,Melvin Gordon,168.0,Brandin Cooks,106.0,Zach Ertz
2016,9,270.0,Blake Bortles,201.0,Jordan Howard,164.0,Michael Thomas,103.0,Jordan Reed
2016,10,268.0,Derek Carr,196.0,Mark Ingram,160.0,Doug Baldwin,95.0,Antonio Gates


For 2023
Maximum total in the bank: 1463.0
Selections that maximize the total:
Index: 0, Vector: rb, Value: 324.0 (Christian McCaffrey)
Index: 1, Vector: wr, Value: 257.0 (Tyreek Hill)
Index: 2, Vector: qb, Value: 343.0 (Dak Prescott)
Index: 3, Vector: wr, Value: 204.0 (Mike Evans) 
Index: 4, Vector: te, Value: 120.0 (David Njoku)
Index: 6, Vector: rb, Value: 215.0 (Breece Hall)


In [98]:
vector_data = %sql $vectors
vector_df = pd.DataFrame(vector_data)
print(vector_df["QB"].tolist())
print(vector_df["RB"].tolist())

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.
[380.0, 347.0, 332.0, 308.0, 300.0, 287.0, 280.0, 271.0, 270.0, 268.0, 268.0, 261.0, 260.0, 259.0, 259.0, 256.0, 254.0, 252.0, 243.0, 242.0, 224.0, 224.0, 221.0, 213.0, 200.0, 192.0, 190.0, 168.0, 133.0, 111.0, 80.0, 77.0, 73.0, 71.0, 66.0, 55.0, 53.0, 52.0, 47.0, 348.0, 299.0, 296.0, 295.0, 282.0, 278.0, 274.0, 270.0, 263.0, 261.0, 261.0, 255.0, 250.0, 238.0, 228.0, 223.0, 211.0, 210.0, 206.0, 206.0, 205.0, 202.0, 187.0, 185.0, 176.0, 169.0, 158.0, 135.0, 130.0, 126.0, 123.0, 102.0, 89.0, 87.0, 74.0, 68.0, 64.0, 62.0, 51.0, 417.0, 354.0, 341.0, 332.0, 327.0, 313.0, 310.0, 304.0, 298.0, 286.0, 285.0, 283.0, 282.0, 280.0, 263.0, 240.0, 240.0, 216.0, 215.0, 212.0, 208.0, 196.0, 193.0, 175.0, 175.0, 173.0, 168.0, 166.0, 158.0, 142.0, 138.0, 137.0, 123.0, 113.0, 85.0, 75.0, 75.0, 68.0, 50.0, 416.0, 338.0, 329.0, 320.0, 305.0, 289.0, 287.0, 285.0, 278.0, 276.0, 267.0, 264.0, 248.0, 247.0, 244.0, 244.0, 241.

In [84]:
import numpy as np

# Define the vectors
vector1 = vector_df["QB"].tolist()
vector2 = vector_df["RB"].tolist()
vector3 = vector_df["WR"].tolist()
vector4 = vector_df["TE"].tolist()

# Number of elements in each vector
n = len(vector1)

# Initialize the DP table
dp = np.full((n + 1, 2, 3, 3, 2), -np.inf)
dp[0][0][0][0][0] = 0

# Initialize the selection table to store choices
choices = np.zeros((n + 1, 2, 3, 3, 2), dtype=int)

# Fill the DP table
for i in range(1, n + 1):
    for v1 in range(2):
        for v2 in range(3):
            for v3 in range(3):
                for v4 in range(2):
                    if dp[i-1][v1][v2][v3][v4] != -np.inf:
                        # Do not select from any vector at index i-1
                        if dp[i][v1][v2][v3][v4] < dp[i-1][v1][v2][v3][v4]:
                            dp[i][v1][v2][v3][v4] = dp[i-1][v1][v2][v3][v4]
                            choices[i][v1][v2][v3][v4] = 0
                        # Select from vector1 at index i-1
                        if v1 < 1 and dp[i][v1+1][v2][v3][v4] < dp[i-1][v1][v2][v3][v4] + vector1[i-1]:
                            dp[i][v1+1][v2][v3][v4] = dp[i-1][v1][v2][v3][v4] + vector1[i-1]
                            choices[i][v1+1][v2][v3][v4] = 1
                        # Select from vector2 at index i-1
                        if v2 < 2 and dp[i][v1][v2+1][v3][v4] < dp[i-1][v1][v2][v3][v4] + vector2[i-1]:
                            dp[i][v1][v2+1][v3][v4] = dp[i-1][v1][v2][v3][v4] + vector2[i-1]
                            choices[i][v1][v2+1][v3][v4] = 2
                        # Select from vector3 at index i-1
                        if v3 < 2 and dp[i][v1][v2][v3+1][v4] < dp[i-1][v1][v2][v3][v4] + vector3[i-1]:
                            dp[i][v1][v2][v3+1][v4] = dp[i-1][v1][v2][v3][v4] + vector3[i-1]
                            choices[i][v1][v2][v3+1][v4] = 3
                        # Select from vector4 at index i-1
                        if v4 < 1 and dp[i][v1][v2][v3][v4+1] < dp[i-1][v1][v2][v3][v4] + vector4[i-1]:
                            dp[i][v1][v2][v3][v4+1] = dp[i-1][v1][v2][v3][v4] + vector4[i-1]
                            choices[i][v1][v2][v3][v4+1] = 4

# Get the maximum value
max_total = max(dp[n][1][2][2][1], 0)

# Trace back the selections
v1, v2, v3, v4 = 1, 2, 2, 1
selections = []

for i in range(n, 0, -1):
    choice = choices[i][v1][v2][v3][v4]
    if choice == 1:
        selections.append((i-1, 'qb', vector1[i-1]))
        v1 -= 1
    elif choice == 2:
        selections.append((i-1, 'rb', vector2[i-1]))
        v2 -= 1
    elif choice == 3:
        selections.append((i-1, 'wr', vector3[i-1]))
        v3 -= 1
    elif choice == 4:
        selections.append((i-1, 'te', vector4[i-1]))
        v4 -= 1

selections.reverse()

print("Maximum total in the bank:", max_total)
print("Selections that maximize the total:")
for selection in selections:
    print(f"Index: {selection[0]}, Vector: {selection[1]}, Value: {selection[2]}")


Maximum total in the bank: 1463.0
Selections that maximize the total:
Index: 0, Vector: rb, Value: 324.0
Index: 1, Vector: wr, Value: 257.0
Index: 2, Vector: qb, Value: 343.0
Index: 3, Vector: wr, Value: 204.0
Index: 4, Vector: te, Value: 120.0
Index: 6, Vector: rb, Value: 215.0


In [99]:
# Loop through each unique year in the dataframe
years = vector_df['YEAR'].unique()
for year in years:
    # Filter dataframe by the current year
    year_df = vector_df[vector_df['YEAR'] == year].reset_index(drop=True)

    # Define the vectors from the filtered dataframe
    vector1 = year_df["QB"].tolist()
    vector2 = year_df["RB"].tolist()
    vector3 = year_df["WR"].tolist()
    vector4 = year_df["TE"].tolist()

    # Number of elements in each vector
    n = len(vector1)

    # Initialize the DP table with -infinity
    dp = np.full((n + 1, 2, 3, 3, 2), -np.inf)
    dp[0][0][0][0][0] = 0

    # Initialize the selection table to store choices
    choices = np.zeros((n + 1, 2, 3, 3, 2), dtype=int)

    # Fill the DP table
    for i in range(1, n + 1):
        for v1 in range(2):
            for v2 in range(3):
                for v3 in range(3):
                    for v4 in range(2):
                        if dp[i-1][v1][v2][v3][v4] != -np.inf:
                            # Do not select from any vector at index i-1
                            if dp[i][v1][v2][v3][v4] < dp[i-1][v1][v2][v3][v4]:
                                dp[i][v1][v2][v3][v4] = dp[i-1][v1][v2][v3][v4]
                                choices[i][v1][v2][v3][v4] = 0
                            # Select from vector1 at index i-1
                            if v1 < 1 and dp[i][v1+1][v2][v3][v4] < dp[i-1][v1][v2][v3][v4] + vector1[i-1]:
                                dp[i][v1+1][v2][v3][v4] = dp[i-1][v1][v2][v3][v4] + vector1[i-1]
                                choices[i][v1+1][v2][v3][v4] = 1
                            # Select from vector2 at index i-1
                            if v2 < 2 and dp[i][v1][v2+1][v3][v4] < dp[i-1][v1][v2][v3][v4] + vector2[i-1]:
                                dp[i][v1][v2+1][v3][v4] = dp[i-1][v1][v2][v3][v4] + vector2[i-1]
                                choices[i][v1][v2+1][v3][v4] = 2
                            # Select from vector3 at index i-1
                            if v3 < 2 and dp[i][v1][v2][v3+1][v4] < dp[i-1][v1][v2][v3][v4] + vector3[i-1]:
                                dp[i][v1][v2][v3+1][v4] = dp[i-1][v1][v2][v3][v4] + vector3[i-1]
                                choices[i][v1][v2][v3+1][v4] = 3
                            # Select from vector4 at index i-1
                            if v4 < 1 and dp[i][v1][v2][v3][v4+1] < dp[i-1][v1][v2][v3][v4] + vector4[i-1]:
                                dp[i][v1][v2][v3][v4+1] = dp[i-1][v1][v2][v3][v4] + vector4[i-1]
                                choices[i][v1][v2][v3][v4+1] = 4

    # Get the maximum value
    max_total = dp[n][1][2][2][1]

    # Trace back the selections
    v1, v2, v3, v4 = 1, 2, 2, 1
    selections = []

    for i in range(n, 0, -1):
        choice = choices[i][v1][v2][v3][v4]
        if choice == 1:
            selections.append((i-1, 'QB', year_df.loc[i-1, 'QB_NAME'], vector1[i-1]))
            v1 -= 1
        elif choice == 2:
            selections.append((i-1, 'RB', year_df.loc[i-1, 'RB_NAME'], vector2[i-1]))
            v2 -= 1
        elif choice == 3:
            selections.append((i-1, 'WR', year_df.loc[i-1, 'WR_NAME'], vector3[i-1]))
            v3 -= 1
        elif choice == 4:
            selections.append((i-1, 'TE', year_df.loc[i-1, 'TE_NAME'], vector4[i-1]))
            v4 -= 1

    selections.reverse()

    # Print the results for the current year
    print(f"\nResults for Year {year}:")
    print("Maximum total in the bank:", max_total)
    print("Selections that maximize the total:")
    for selection in selections:
        print(f"Index: {selection[0]}, Position: {selection[1]}, Name: {selection[2]}, Value: {selection[3]}")



Results for Year 2016:
Maximum total in the bank: 1449.0
Selections that maximize the total:
Index: 0, Position: RB, Name: David Johnson, Value: 328.0
Index: 1, Position: RB, Name: Ezekiel Elliott, Value: 293.0
Index: 2, Position: QB, Name: Drew Brees, Value: 332.0
Index: 3, Position: WR, Name: Odell Beckham Jr., Value: 196.0
Index: 4, Position: TE, Name: Delanie Walker, Value: 123.0
Index: 5, Position: WR, Name: Julio Jones, Value: 177.0

Results for Year 2017:
Maximum total in the bank: 1345.0
Selections that maximize the total:
Index: 0, Position: RB, Name: Todd Gurley, Value: 319.0
Index: 1, Position: WR, Name: Antonio Brown, Value: 209.0
Index: 2, Position: TE, Name: Zach Ertz, Value: 128.0
Index: 3, Position: QB, Name: Alex Smith, Value: 295.0
Index: 4, Position: RB, Name: Melvin Gordon, Value: 230.0
Index: 5, Position: WR, Name: Julio Jones, Value: 164.0

Results for Year 2018:
Maximum total in the bank: 1566.0
Selections that maximize the total:
Index: 0, Position: QB, Name: P

In [103]:
# Number of players to remove from each vector after each selection
removal_counts = {
    'QB': 2,
    'RB': 3,
    'WR': 3,
    'TE': 1
}

# Loop through each unique year in the dataframe
years = vector_df['YEAR'].unique()
for year in years:
    # Filter dataframe by the current year
    year_df = vector_df[vector_df['YEAR'] == year].reset_index(drop=True)

    # Define the vectors from the filtered dataframe
    vector1 = year_df["QB"].tolist()
    vector2 = year_df["RB"].tolist()
    vector3 = year_df["WR"].tolist()
    vector4 = year_df["TE"].tolist()

    # Number of elements in each vector
    n = len(vector1)

    # Initialize the DP table with -infinity
    dp = np.full((n + 1, 2, 3, 3, 2), -np.inf)
    dp[0][0][0][0][0] = 0

    # Initialize the selection table to store choices
    choices = np.zeros((n + 1, 2, 3, 3, 2), dtype=int)

    # Initialize removal queue
    removal_queue = deque()
    for pos in ['QB', 'RB', 'WR', 'TE']:
        for _ in range(removal_counts[pos]):
            removal_queue.append(pos)

    # Fill the DP table
    v1_idx, v2_idx, v3_idx, v4_idx = 0, 0, 0, 0
    for i in range(1, n + 1):
        for v1 in range(2):
            for v2 in range(3):
                for v3 in range(3):
                    for v4 in range(2):
                        if dp[i-1][v1][v2][v3][v4] != -np.inf:
                            # Do not select from any vector at index i-1
                            if dp[i][v1][v2][v3][v4] < dp[i-1][v1][v2][v3][v4]:
                                dp[i][v1][v2][v3][v4] = dp[i-1][v1][v2][v3][v4]
                                choices[i][v1][v2][v3][v4] = 0
                            # Select from vector1 at index i-1
                            if v1 < 1 and v1_idx < len(vector1) and dp[i][v1+1][v2][v3][v4] < dp[i-1][v1][v2][v3][v4] + vector1[v1_idx]:
                                dp[i][v1+1][v2][v3][v4] = dp[i-1][v1][v2][v3][v4] + vector1[v1_idx]
                                choices[i][v1+1][v2][v3][v4] = 1
                            # Select from vector2 at index i-1
                            if v2 < 2 and v2_idx < len(vector2) and dp[i][v1][v2+1][v3][v4] < dp[i-1][v1][v2][v3][v4] + vector2[v2_idx]:
                                dp[i][v1][v2+1][v3][v4] = dp[i-1][v1][v2][v3][v4] + vector2[v2_idx]
                                choices[i][v1][v2+1][v3][v4] = 2
                            # Select from vector3 at index i-1
                            if v3 < 2 and v3_idx < len(vector3) and dp[i][v1][v2][v3+1][v4] < dp[i-1][v1][v2][v3][v4] + vector3[v3_idx]:
                                dp[i][v1][v2][v3+1][v4] = dp[i-1][v1][v2][v3][v4] + vector3[v3_idx]
                                choices[i][v1][v2][v3+1][v4] = 3
                            # Select from vector4 at index i-1
                            if v4 < 1 and v4_idx < len(vector4) and dp[i][v1][v2][v3][v4+1] < dp[i-1][v1][v2][v3][v4] + vector4[v4_idx]:
                                dp[i][v1][v2][v3][v4+1] = dp[i-1][v1][v2][v3][v4] + vector4[v4_idx]
                                choices[i][v1][v2][v3][v4+1] = 4

        # Update indices after selecting a player
        if i < n:
            pos_to_remove = removal_queue.popleft()
            if pos_to_remove == 'QB':
                v1_idx += 1
            elif pos_to_remove == 'RB':
                v2_idx += 1
            elif pos_to_remove == 'WR':
                v3_idx += 1
            elif pos_to_remove == 'TE':
                v4_idx += 1

    # Get the maximum value
    max_total = dp[n][1][2][2][1]

    # Trace back the selections
    v1, v2, v3, v4 = 1, 2, 2, 1
    selections = []

    for i in range(n, 0, -1):
        choice = choices[i][v1][v2][v3][v4]
        if choice == 1:
            selections.append((i-1, 'QB', year_df.loc[i-1, 'QB_NAME'], vector1[v1_idx-1]))
            v1 -= 1
            v1_idx -= 1
        elif choice == 2:
            selections.append((i-1, 'RB', year_df.loc[i-1, 'RB_NAME'], vector2[v2_idx-1]))
            v2 -= 1
            v2_idx -= 1
        elif choice == 3:
            selections.append((i-1, 'WR', year_df.loc[i-1, 'WR_NAME'], vector3[v3_idx-1]))
            v3 -= 1
            v3_idx -= 1
        elif choice == 4:
            selections.append((i-1, 'TE', year_df.loc[i-1, 'TE_NAME'], vector4[v4_idx-1]))
            v4 -= 1
            v4_idx -= 1

    selections.reverse()

    # Print the results for the current year
    print(f"\nResults for Year {year}:")
    print("Maximum total in the bank:", max_total)
    print("Selections that maximize the total:")
    for selection in selections:
        print(f"Index: {selection[0]}, Position: {selection[1]}, Name: {selection[2]}, Value: {selection[3]}")


IndexError: pop from an empty deque

In [104]:
import requests
import json

url = "http://localhost:11434/api/chat"

def llama3(prompt):
    data = {
        "model": "llama3",
        "messages": [
            {
              "role": "user",
              "content": prompt
            }
        ],
        "stream": False
    }
    
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    return(response.json()['message']['content'])

/Users/beoconno/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
